# ICE-NMF Evaluation ymusic

In [7]:
#imports
import numpy as np 
import csv
import re
import sys
import math
from numpy import linalg as L
import pandas as pd
import random
from sklearn.preprocessing import normalize
from numba import njit
from collections import defaultdict
import random
from implicit_data import ImplicitData
import pandas as pd
from BISGD import BISGD
from ICE_NMF import ICE_NMF
from collections import defaultdict
from EvalPrequential import EvalPrequential
from datetime import datetime
import getopt

## Functions


In [8]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

def sumvector(V,x):
    result = []
    for v in V:
        temp = v + x
        result.append(temp)
    return result    

def multvector(x,V):
    result = []
    for v in V:
        temp = v * x
        result.append(temp)
    return result    

def TransformVec(V):
    for i in range(len(V)):
        V[i] = sigmoid(V[i])
    D = LA.norm(V)
    D=1/D
    new = D * V
  

    return new


## Get the data

In [9]:
sys.path.append('./')
configPath = 'config'
dataPath = 'ymusic_gte90_6kusers.tsv'


Data = []
data = pd.read_csv("ymusic_gte90_6kusers.tsv","\t")
data = data.head(n=46000)
print(data)
Data = data.values.tolist()



       user_id  item_id
0       378723   124036
1       378723   608145
2       378723    48098
3       378723    80338
4       378723   334716
...        ...      ...
45995   295010   506081
45996   382938    74722
45997   221033   158827
45998   221033   603920
45999   221033    26129

[46000 rows x 2 columns]


# Full dataset

In [14]:

Data_Transpose = np.transpose(Data[:50000])
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])

num_clusters = {8,16,24}
cl_num_iterations = 10
cl_learn_rate = 0.1
cl_regularization = 0.1
num_factors = 200
num_iter = 9
learn_rate = 0.25
regularization = 0.45
overlap = {0.6,0.7,0.8}


In [15]:
%%capture out

tuning = {}

for num in num_clusters:
    tuning[num] = {}
    
    
    for val in overlap:

        print("")
        print("num_clusters = ", num, ", overlap = ", val)
        model = ICE_NMF(ImplicitData([],[]), num, val, cl_num_iterations, cl_learn_rate, cl_regularization, 
        num_factors, num_iter, learn_rate, regularization, random_seed = 100)
        eval = EvalPrequential(model,stream, metrics = ["Recall@20"])

        start_recommend = datetime.now()
        print('start time', start_recommend)

        results=eval.EvaluateTime(0,stream.size, 100)

        end_recommend = datetime.now()
        print('end time', end_recommend)

        tempo = end_recommend - start_recommend

        tuning[num][val] = np.mean(results['Recall@20'])
    
        print('npmean(resuls[Recall@20])', tuning[num][val])
        print('run time', tempo)
        #print('get tuple',np.mean(results['time_get_tuple']))
        print('recommend',np.mean(results['time_recommend']))
        #print('eval_point',np.mean(results['time_eval_point']))
        print('update',np.mean(results['time_update'])) 

In [16]:
print(out)


num_clusters =  8 , overlap =  0.6
start time 2023-11-23 14:06:09.996028
end time 2023-11-23 14:07:52.608964
npmean(resuls[Recall@20]) 0.16556291390728478
run time 0:01:42.612936
recommend 0.09387526396357723
update 0.0013005809317464415

num_clusters =  8 , overlap =  0.7
start time 2023-11-23 14:07:52.701231
end time 2023-11-23 14:09:45.582559
npmean(resuls[Recall@20]) 0.2032520325203252
run time 0:01:52.881328
recommend 0.0940533731041885
update 0.0014421815872192384

num_clusters =  8 , overlap =  0.8
start time 2023-11-23 14:09:45.672739
end time 2023-11-23 14:11:38.409878
npmean(resuls[Recall@20]) 0.17142857142857143
run time 0:01:52.737139
recommend 0.09521657933994215
update 0.00143066478293875

num_clusters =  16 , overlap =  0.6
start time 2023-11-23 14:11:38.494507
end time 2023-11-23 14:14:35.146337
npmean(resuls[Recall@20]) 0.17647058823529413
run time 0:02:56.651830
recommend 0.19570797471439136
update 0.0020263982700264973

num_clusters =  16 , overlap =  0.7
start time

In [17]:
tuning

{8: {0.6: 0.16556291390728478,
  0.7: 0.2032520325203252,
  0.8: 0.17142857142857143},
 16: {0.6: 0.17647058823529413,
  0.7: 0.19724770642201836,
  0.8: 0.17721518987341772},
 24: {0.6: 0.16973415132924335,
  0.7: 0.20964360587002095,
  0.8: 0.20090293453724606}}